# The probe phase
There are several ways of analyzing the topology and trajectory pairs, depending on the number of trajectory files per a topology file, the continuity of trjaectory files, organization of files in a directory, and the parallel or sequencial arrangement of the computation powerhorse.


### To-do-list

This notebook is a living documnent. It is an integration of all the past notebooks and scripts written for the *probe* phase in the *PolyPhys* package (or formerly-called *extraction* phase in the decryped *sumrule* package)

BThe list below allows to review the past notebooks and scripts and combine them into this notebook:

- [] Sum-rule: segments: bug test and run on **date:NODATE-YET**
- [] Sum-rule: segments: all test and run on **date:NODATE-YET**
- [] Sum-rule: wholes: bug test and run on **date:NODATE-YET**
- [] Sum-rule: wholes: all test and run on **date:NODATE-YET**
- [] Sum-rule: all_in_one: test and run on **date:NODATE-YET**
- [X] Trans-Foci: segments: bug test and run on **date:20220621**
- [] Trans-Foci: segments: all test and run on **date:NODATE-YET**
- [] Trans-Foci: wholes: bug test and run on **date:NODATE-YET**
- [] Trans-Foci: wholes: all test and run on **date:NODATE-YET**
- [] Trans-Foci: all_in_one: test and run on **date:NODATE-YET**

### Naming convention:

This is the pattern of file or directory names:

1. **whole** files: whole-group-property_[-measure][-stage][.ext]
2. **ensemble** files: ensemble-group-property_[-measure][-stage][.ext]
3. **ensemble_long** files: ensemble_long-group-property_[-measure][-stage][.ext]
4. **space** files: space-group-property_[-measure][-stage][.ext]
5. **all in one** files: space-group-**species**-**allInOne**-property-_[-measure][-stage][.ext]

[keyword] means that the keyword in the file name is option. [-measure] is a physical measurement such as the auto correlation function (AFC) done on the physical 'property_'.

### Settings for testing and running on a PC.

In [ ]:
# settings for testing and running on a PC.
from glob import glob
from polyphys.manage import organizer
from polyphys.manage.parser import \
    SumRuleCyl, TransFociCyl, TransFociCub, HnsCub
from polyphys.probe import prober

# HPC Cluster: gnuparallel

## Separated *whole* simulation directories

On a cluster, *whole* simulations are organized into *whole* directories, where each *whole* directory contains all the files for a given *whole* simulation. The **gnuparallel** is used to parallalize the **probe** phase at the **shell** level. For this purpose, all the python modules and scripts are separatedly installed and run on each core. For instance, if 32 cores are available, then the files in 32 *whole* directories are simulatenously installed. However, each *whole* directory may contains multiple toplogy and trajectory pairs. Thus, there is parallelization at the level of *whole* directories, not at the levle of the *segment* or *whole* trajectories inside a *whole* directory. Inside each *whole* directory, a python **main_probe.py** script analyzes the trajectories in a sequencial way.

- trj and all *segments* on a cluster

For each *whole* directory, the following script is executed by means of *gnuparallel*. See these scripts: *probe-1.7-all_trj_segments.py* and *probe-1.7-bug_trj_segments*

## Sum-rule project

#### whole bug trjs

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober


# analyzing bug files.
group = 'bug'
lineage = 'whole'
save_to = './'
bug_pairs = glob('./N*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.sum_rule_bug_cyl(bug_topo, bug_trj, lineage, save_to=save_to)

#### segment bug trjs

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober
from polyphys.manage.parser import SumRuleCyl


group = 'bug'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
bug_trjs = glob('./N*' + group + '*')
bug_trjs = organizer.sort_filenames(
    bug_trjs,
    fmts=['.' + group + '.lammpstrj']
)
bug_trjs = [bug_trj[0] for bug_trj in bug_trjs]
bug_topo = glob('./N*' + group + '*')
bug_topo = organizer.sort_filenames(bug_topo, fmts=['.' + group + '.data'])
bug_topo = bug_topo[0][0]
max_segment_id = len(bug_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for bug_trj in bug_trjs:
    trj_info = SumRuleCyl(bug_trj, topo_lineage, 'cylindrical', group)
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_bug_cyl(bug_topo, bug_trj, lineage, save_to=save_to)
    else:
        prober.sum_rule_bug_cyl(
            bug_topo,
            bug_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### segment all trjs

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober
from polyphys.manage.parser import SumRuleCyl


group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob('./N*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob('./N*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = SumRuleCyl(all_trj, topo_lineage, 'cylindrical', group)
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_all_cyl(all_topo, all_trj, lineage, save_to=save_to)
    else:
        prober.sum_rule_all_cyl(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### whole bug trjs:  transverse size

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober


# analyzing bug files.
group = 'bug'
lineage = 'whole'
save_to = './'
bug_pairs = glob('./N*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.sum_rule_bug_cyl_transverse_size(
        bug_topo,
        bug_trj,
        lineage,
        save_to = save_to
    )

#### segment bug trjs: transverse size

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober
from polyphys.manage.parser import SumRuleCyl

group = 'bug'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
bug_trjs = glob('./N*' + group + '*')
bug_trjs = organizer.sort_filenames(
    bug_trjs,
    fmts=['.' + group + '.lammpstrj']
)
bug_trjs = [bug_trj[0] for bug_trj in bug_trjs]
bug_topo = glob('./N*' + group + '*')
bug_topo = organizer.sort_filenames(bug_topo, fmts=['.' + group + '.data'])
bug_topo = bug_topo[0][0]
max_segment_id = len(bug_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for bug_trj in bug_trjs:
    trj_info = SumRuleCyl(bug_trj, topo_lineage, 'cylindrical', group)
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_bug_cyl_transverse_size(
            bug_topo, bug_trj, lineage, save_to=save_to
        )
    else:
        prober.sum_rule_bug_cyl_transverse_size(
            bug_topo,
            bug_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### segment all trjs: hist_dd

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober
from polyphys.manage.parser import SumRuleCyl


geometry = 'biaxial'
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob('./N*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob('./N*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = SumRuleCyl(all_trj, topo_lineage, 'cylindrical', group)
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_all_cyl_hist2d(
            all_topo, all_trj, lineage, save_to=save_to
        )
    else:
        prober.sum_rule_all_cyl_hist2d(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

## Trans-Foci project

### Cylindrical

#### whole bug trjs
Each *bug topology* comes with only **one** *bug trajectory*.

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober

# analyzing bug files.
group = 'bug'
lineage = 'whole'
save_to = './'
bug_pairs = glob('./eps*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.trans_fuci_bug_cyl(bug_topo, bug_trj, lineage, save_to=save_to)

#### segment all trjs

In [ ]:
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob('./eps*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob('./eps*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = TransFociCyl(all_trj, topo_lineage, 'cylindrical', group)
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.trans_foci_all_cyl(all_topo, all_trj, lineage, save_to=save_to)
    else:
        prober.trans_foci_all_cyl(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### whole bug trjs: transverse_size
Each *bug topology* comes with only **one** *bug trajectory*.

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober

# analyzing bug files.
group = 'bug'
lineage = 'whole'
save_to = './'
bug_pairs = glob('./eps*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.trans_fuci_bug_cyl_transverse_size(
        bug_topo, bug_trj, lineage, save_to=save_to
    )

#### segment all trjs: hist_dd

In [ ]:
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob('./eps*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob('./eps*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = TransFociCyl(
        all_trj,
        topo_lineage,
        'cylindrical',
        group
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.trans_foci_all_cyl_hist2d(
            all_topo, all_trj, lineage, save_to=save_to
        )
    else:

        prober.trans_foci_all_cyl_hist2d(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

### Cubic

#### whole bug trjs
Each *bug topology* comes with only **one** *bug trajectory*.

In [ ]:
from glob import glob
from polyphys.manage import organizer
from polyphys.probe import prober

# analyzing bug files.
geometry = 'biaxial'
group='bug'
lineage = 'whole'
save_to = './'
bug_pairs = glob('./al*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.trans_fuci_bug_cubic(bug_topo, bug_trj, lineage, save_to=save_to)

#### segment all trjs

In [ ]:
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob('./al*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob('./al*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = TransFociCubic(all_trj, topo_lineage, 'cubic', group)
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.trans_foci_all_cubic(
            all_topo, all_trj, lineage, save_to=save_to
        )
    else:
        prober.trans_foci_all_cubic(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

# PC Serial scheme 

There are 4 different types of directories from which only one type can be in a *space* directory.

There are separated **whole** directories in each of which there **all** and **bug** **whole** trajectories; or, there are again separated **whole** directories in each of which there are **all** and **bug** **segment** trjaectories. Below there are two groups of scrips for **serial** and **parallel** runnung schemes.


On a PC, the *whole* directories are located in a master *space-trjs* directory; however, one main python script probes all the *whole* directories in a parallel scheme via Dask. This is different from the *gnuparallel*-based approach in which each *whole* directory has its own copy of the required scripts and a main pytohn script is run to probe that direcotry individually.

## Sumrule project

### Cylindrical

#### whole bug trjs

In [ ]:
# analyzing bug files.
group = 'bug'
lineage = 'whole'
save_to = './'
macmini_path = "/Users/amirhsi_mini/trjs/epss5.0epsl5.0r10.5al5.0nl5ml125ns200ac1.0nc*lz77.0dt0.005bdump5000adump5000ens1ring/*.bug*"
#macbookpro_path = "/Users/amirhsi/Downloads/N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7"
bug_pairs = glob(macbookpro_path + '/N*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.sum_rule_bug_cyl(
        bug_topo,
        bug_trj,
        lineage,
        continuous=False,
        save_to = save_to
    )

#### segment bug trjs

In [ ]:
# 1 min and 45 s for ~30000 particles with one all trj
macbookpro_path = "/Users/amirhsi/Downloads/N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7"
group = 'bug'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
bug_trjs = glob(macbookpro_path + '/N*' + group + '*')
bug_trjs = organizer.sort_filenames(
    bug_trjs,
    fmts=['.' + group + '.lammpstrj']
)
bug_trjs = [bug_trj[0] for bug_trj in bug_trjs]
bug_topo = glob(macbookpro_path + '/N*' + group + '*')
bug_topo = organizer.sort_filenames(bug_topo, fmts=['.' + group + '.data'])
bug_topo = bug_topo[0][0]
max_segment_id = len(bug_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for bug_trj in bug_trjs:
    trj_info = SumRuleCyl(
        bug_trj,
        topo_lineage,
        'cylindrical',
        group,
        'linear'
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_bug_cyl(
            bug_topo, bug_trj, lineage, save_to=save_to
        )
    else:
        prober.sum_rule_bug_cyl(
            bug_topo,
            bug_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### segment all trjs: hist_1d and hist_2d

In [ ]:
# 4 mins for ~4000 particles with one all trj
# macbookmini
parent = "/Users/amirhsi_mini/research_data/trjs"
# macbookpro
#parent = "/Users/amirhsi/Downloads/N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7"
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob(parent + '/N*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
print(all_trjs)
all_topo = glob(parent + '/N*' + group + '*')
print(all_topo)
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = SumRuleCyl(
        all_trj,
        topo_lineage,
        'cylindrical',
        group,
        'linear'
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_all_cyl(
            all_topo, all_trj, lineage, save_to=save_to
        )
    else:
        prober.sum_rule_all_cyl(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### whole bug trjs: transverse size

In [ ]:
# analyzing bug files.
group = 'bug'
lineage = 'whole'
save_to = './'
macmini_path = "/Users/amirhsi_mini/trjs/N2000epsilon5.0r15.5lz379.5sig4.0nc4804dt0.005bdump1000adump5000ens6/*.bug*"
#macbookpro_path = "/Users/amirhsi/Downloads/N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7"
bug_pairs = glob(macbookpro_path + '/N*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.sum_rule_bug_cyl_transverse_size(
        bug_topo,
        bug_trj,
        lineage,
        continuous=False,
        save_to = save_to
    )

#### segment bug trjs: transverse size

In [ ]:
%%time
# 18 mins for N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7 two bugs
#trjs_db = "/Users/amirhsi/Downloads/N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7"
trjs_db = "/Users/amirhsi_mini/research_data/trjs/N2000epsilon5.0r15.5lz379.5sig4.0nc4804dt0.005bdump1000adump5000ens6"
group = 'bug'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
bug_trjs = glob(trjs_db + '/N*' + group + '*')
bug_trjs = organizer.sort_filenames(
    bug_trjs,
    fmts=['.' + group + '.lammpstrj']
)
bug_trjs = [bug_trj[0] for bug_trj in bug_trjs]
bug_topo = glob(trjs_db + '/N*' + group + '*')
bug_topo = organizer.sort_filenames(bug_topo, fmts=['.' + group + '.data'])
bug_topo = bug_topo[0][0]
max_segment_id = len(bug_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for bug_trj in bug_trjs:
    trj_info = SumRuleCyl(
        bug_trj,
        topo_lineage,
        'cylindrical',
        group,
        'linear'
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_bug_cyl_transverse_size(
            bug_topo, bug_trj, lineage, save_to=save_to
        )
    else:
        prober.sum_rule_bug_cyl_transverse_size(
            bug_topo,
            bug_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### segment all trjs: hist_2d

In [ ]:
%%time
# 10 mins for N2000epsilon5.0r13.0lz259.0sig1.0nc133547dt0.005bdump1000adump5000ens1 2 all trjs
# macbookmini
trjs_db = "/Users/amirhsi_mini/research_data/trjs/N2000epsilon5.0r13.0lz259.0sig1.0nc133547dt0.005bdump1000adump5000ens1"
# macbookpro
#trjs_db = "/Users/amirhsi/Downloads/N2000epsilon5.0r15.5lz379.5sig6.0nc1779dt0.005bdump1000adump5000ens7"
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob(trjs_db + '/N*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
print(all_trjs)
all_topo = glob(trjs_db + '/N*' + group + '*')
print(all_topo)
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = SumRuleCyl(
        all_trj,
        topo_lineage,
        'cylindrical',
        group,
        'linear'
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.sum_rule_all_cyl_hist2d(
            all_topo, all_trj, lineage, save_to=save_to
        )
        
    else:
        prober.sum_rule_all_cyl_hist2d(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

## Trans-Foci project

### Cylindrical

#### whole bug trjs

In [ ]:
# analyzing bug files.
group = 'bug'
linage = 'whole'
save_to = './'
#macmini_path = "/Users/amirhsi_mini/trjs/epss5.0epsl5.0r10.5al5.0nl5ml125ns200ac1.0nc*lz77.0dt0.005bdump5000adump5000ens1ring/*.bug*"
macbookpro_path = "/Users/amirhsi/Downloads/epss5epsl5r10.5al5nl5ml125ns400ac1nc27720lz77dt0.005bdump2000adump5000ens8"
bug_pairs = glob(macbookpro_path + '/eps*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.trans_fuci_bug_cyl(
        bug_topo,
        bug_trj,
        lineage,
        save_to = save_to
    )

#### segment all trjs

In [ ]:
# 1 min and 45 s for ~30000 particles with one all trj
#macbookpro_path = "/Users/amirhsi/Downloads/epss5epsl5r10.5al5nl5ml125ns400ac1nc27720lz77dt0.005bdump2000adump5000ens8"
macmini_path = '/Users/amirhsi_mini/research_data/test'
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob(macmini_path + '/eps*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob(macmini_path + '/eps*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = TransFociCyl(
        all_trj,
        topo_lineage,
        'cylindrical',
        group,
        'ring'
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.trans_foci_all_cyl(
            all_topo, all_trj, lineage, save_to=save_to
        )
    else:
        prober.trans_foci_all_cyl(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

#### segment bug trjs: transverse size

In [ ]:
# analyzing bug files.
group = 'bug'
linage = 'whole'
save_to = './'
#macmini_path = "/Users/amirhsi_mini/trjs/epss5.0epsl5.0r10.5al5.0nl5ml125ns200ac1.0nc*lz77.0dt0.005bdump5000adump5000ens1ring/*.bug*"
macbookpro_path = "/Users/amirhsi/Downloads/epss5epsl5r10.5al5nl5ml125ns400ac1nc27720lz77dt0.005bdump2000adump5000ens8"
bug_pairs = glob(macbookpro_path + '/eps*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.trans_foci_bug_cyl_transverse_size(
        bug_topo,
        bug_trj,
        lineage,
        save_to = save_to
    )

#### segment all trjs: hist_2d

In [ ]:
# 1 min and 45 s for ~30000 particles with one all trj
#macbookpro_path = "/Users/amirhsi/Downloads/epss5epsl5r10.5al5nl5ml125ns400ac1nc27720lz77dt0.005bdump2000adump5000ens8"
macmini_path = '/Users/amirhsi_mini/research_data/test'
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob(macmini_path + '/eps*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob(macmini_path + '/eps*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = TransFociCyl(
        all_trj,
        topo_lineage,
        'cylindrical',
        group,
        'ring'
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.trans_foci_all_hist2d(
            all_topo, all_trj, lineage, save_to=save_to
        )
    else:
        prober.trans_foci_all_hist2d(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

### Cubic

#### whole bug trjs

In [ ]:
# analyzing bug files.
group = 'bug'
lineage = 'whole'
save_to = './'
#macmini_path = "/Users/amirhsi_mini/trjs/epss5.0epsl5.0r10.5al5.0nl5ml125ns200ac1.0nc*lz77.0dt0.005bdump5000adump5000ens1ring/*.bug*"
macbookpro_path = "/Users/amirhsi_mini/research_data/ns400nl5al5ml125ac1nc0l62dt0.005bdump2000adump5000ens3.ring"
bug_pairs = glob(macbookpro_path + '/al*' + group + '*')
bug_pairs = organizer.sort_filenames(
    bug_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (bug_topo, bug_trj) in bug_pairs:
    prober.trans_fuci_bug_cub(
        bug_topo,
        bug_trj,
        lineage,
        save_to = save_to
    )

#### segment all trjs

In [ ]:
%%time
# 1 min and 45 s for ~30000 particles with one all trj
macbookpro_path = "/Users/amirhsi_mini/research_data/ns400nl5al5ml125ac1nc0l62dt0.005bdump2000adump5000ens3.ring"
#macmini_path = '/Users/amirhsi_mini/research_data/test'
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob(macbookpro_path + '/al*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob(macbookpro_path + '/al*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = TransFociCub(
        all_trj,
        topo_lineage,
        'cubic',
        group,
        'ring'
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.trans_foci_all_cub(
            all_topo, all_trj, lineage, save_to=save_to
        )
    else:
        prober.trans_foci_all_cub(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

## HNS project

### Cubic

#### whole bug trjs

In [ ]:
# analyzing bug files.
group = 'nucleoid'
lineage = 'whole'
save_to = './'
probe_path = "/Users/amirhsi_mini/research_data/hns_cubic-trjs/N*/N*"
#probe_path = "/Users/amirhsi_mini/research_data/ns400nl5al5ml125ac1nc0l62dt0.005bdump2000adump5000ens3.ring"
nuc_pairs = glob(probe_path + '/N*' + group + '*')
nuc_pairs = organizer.sort_filenames(
    nuc_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (nuc_topo, nuc_trj) in nuc_pairs:
    prober.hns_nucleoid_cub(
        nuc_topo,
        nuc_trj,
        lineage,
        save_to = save_to
    )

#### segment all trjs

In [ ]:
%%time
# 1 min and 45 s for ~30000 particles with one all trj
macbookpro_path = "/Users/amirhsi_mini/research_data/ns400nl5al5ml125ac1nc0l62dt0.005bdump2000adump5000ens3.ring"
#macmini_path = '/Users/amirhsi_mini/research_data/test'
group = 'all'
topo_lineage = 'whole'
lineage = 'segment'
save_to = './'
all_trjs = glob(macbookpro_path + '/al*' + group + '*')
all_trjs = organizer.sort_filenames(
    all_trjs,
    fmts=['.' + group + '.lammpstrj']
)
all_trjs = [all_trj[0] for all_trj in all_trjs]
all_topo = glob(macbookpro_path + '/al*' + group + '*')
all_topo = organizer.sort_filenames(all_topo, fmts=['.' + group + '.data'])
all_topo = all_topo[0][0]
max_segment_id = len(all_trjs)
# analyzig all files
# it is assumed that the all trjs are numbers from 1 to max_segment_id
for all_trj in all_trjs:
    trj_info = HnsCub(
        all_trj,
        topo_lineage,
        'cubic',
        group,
        'ring'
    )
    # all the frames in the last segment are probed:
    if trj_info.segment_id == max_segment_id:
        prober.hns_all_cub(
            all_topo, all_trj, lineage, save_to=save_to
        )
    else:
        prober.hns_all_cub(
            all_topo,
            all_trj,
            lineage,
            save_to=save_to,
            continuous=True
        )

### Hns Cubi: test probe

In [ ]:
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from typing import Optional, Tuple, Dict
import MDAnalysis as mda
from MDAnalysis.analysis.base import AnalysisFromFunction
import numpy as np
import pandas as pd
import os
from polyphys.manage.typer import ParserT
from polyphys.manage.parser import SumRuleCyl, TransFociCyl, TransFociCub
from polyphys.manage.organizer import invalid_keyword, sort_filenames
from polyphys.analyze import clusters, correlations
from polyphys.probe.prober import stamps_report
import warnings
from glob import glob

from MDAnalysis.analysis import polymer
from MDAnalysis.analysis import distances
from polyphys.manage.parser import HnsCub

In [ ]:
def hns_nucleoid_cub_dist(
    topology: str,
    trajectory: str,
    lineage: str,
    save_to: str = './',
    continuous: bool = False
) -> None:
    if (lineage == 'segment') & (continuous is False):
        warnings.warn(
            "lineage is "
            f"'{lineage}' "
            "and 'continuous' is "
            f"'{continuous}. "
            "Please ensure the "
            f"'{trajectory}' is NOT part of a sequence of trajectories.",
            UserWarning
        )
    print("Setting the name of analyze file...")
    sim_info = HnsCub(
        trajectory,
        lineage,
        'cubic',
        'nucleoid',
        'ring'
    )
    sim_name = sim_info.lineage_name + "-" + sim_info.group
    print("\n" + sim_name + " is analyzing...\n")
    # LJ time difference between two consecutive frames:
    time_unit = sim_info.dmon * np.sqrt(
        sim_info.mmon * sim_info.eps_others)  # LJ time unit
    lj_nstep = sim_info.ndump  # Sampling steps via dump command in Lammps
    lj_dt = sim_info.dt
    sim_real_dt = lj_nstep * lj_dt * time_unit
    cell = mda.Universe(
        topology, trajectory, topology_format='DATA',
        format='LAMMPSDUMP', lammps_coordinate_convention='unscaled',
        atom_style="id resid type x y z", dt=sim_real_dt
        )
    # slicing trajectory based the continuous condition
    if continuous:
        sliced_trj = cell.trajectory[0: -1]
        n_frames = cell.trajectory.n_frames - 1
    else:
        sliced_trj = cell.trajectory
        n_frames = cell.trajectory.n_frames
    # selecting atom groups
    bug = cell.select_atoms('resid 1')  # the bug
    hns_pole = cell.select_atoms('type 2')  # the hns holes
    hns_core = cell.select_atoms('type 3')  # the hns cores

    # bond info
    n_bonds = len(bug.bonds.indices)
    bond_lengths = np.zeros(n_bonds, dtype=np.float64)
    cosine_corrs = np.zeros(n_bonds, dtype=np.float64)
    #ljcut_coeff = 1.122462
    #hns_core_mon_cutoff = 0.5*(
    #    sim_info.dmon*ljcut_coeff + 0.2 + sim_info.dhns
    #)
    hns_pole_mon_cutoff = 0.5*(sim_info.dmon*ljcut_coeff + 0.2)
    #mon_mon_cutoff = sim_info.dmon*ljcut_coeff + 0.2 + sim_info.dhns
    dist_array_core_contact = np.zeros((sim_info.nmon, sim_info.nhns), dtype=np.int64)
    dist_array_pole_contact = np.zeros((sim_info.nmon, 2*sim_info.nhns), dtype=np.int64)
    dist_array_mon_contact = np.zeros((sim_info.nmon, sim_info.nmon), dtype=np.int64)
    for _ in sliced_trj:
        # bug:
        #dummy = distances.distance_array(bug, hns_core, box=cell.dimensions)
        #dummy = np.asarray(dummy <= hns_core_mon_cutoff, dtype=int)
        #dist_array_core_contact += dummy
        dummy = distances.distance_array(bug, hns_pole, box=cell.dimensions)
        dummy = np.asarray(dummy <= hns_pole_mon_cutoff, dtype=int)
        dist_array_pole_contact += dummy
        #dummy = distances.distance_array(bug, bug, box=cell.dimensions)
        #dummy = np.asarray(dummy <= mon_mon_cutoff, dtype=int)
        #dist_array_mon_contact += dummy
        # bug:
        #bond_dummy, cosine_dummy = correlations.bond_info(bug.positions, 'ring')
        #bond_lengths += bond_dummy.reshape(200)
        #cosine_corrs += cosine_dummy

    #dist_array_core_contact_mean = dist_array_core_contact / n_frames
    dist_array_pole_contact_mean = dist_array_pole_contact / n_frames
    #dist_array_mon_contact_mean = dist_array_mon_contact / n_frames
    #bond_lengths_mean = bond_lengths_acc / n_frames
    #cosine_corrs_mean = cosine_corrs_acc / n_frames
    #np.save(
    #    save_to + sim_name + "distCoreMonMean.npy",
    #    dist_array_core_contact_mean
    #)
    np.save(
        save_to + sim_name + "-distPoleMonMean.npy",
        dist_array_pole_contact_mean
    )
    #np.save(
    #    save_to + sim_name + "-distMonMonMeanNewCutoff.npy",
    #    dist_array_mon_contact_mean
    #)
    bond_lengths = bond_lengths / n_frames
    bonds_per_lag = np.arange(n_bonds, 0, -1)
    cosine_corrs = cosine_corrs / (n_frames * bonds_per_lag)
    np.save(save_to + sim_name +'-bondLengthVec.npy', bond_lengths)
    np.save(save_to + sim_name +'-bondCosineCorrVec.npy', cosine_corrs)

In [ ]:
%%time
# 2min per pairs
# 32 min for 10 pairs
# analyzing bug files.
group = 'nucleoid'
lineage = 'whole'
save_to = './'
probe_path = "/Users/amirhsi_mini/research_data/hns_cubic-trjs/N*/N200epshm29nh49ac1*"
#probe_path = "/Users/amirhsi_mini/research_data/ns400nl5al5ml125ac1nc0l62dt0.005bdump2000adump5000ens3.ring"
nuc_pairs = glob(probe_path + '/N*' + group + '*')
nuc_pairs = organizer.sort_filenames(
    nuc_pairs,
    fmts=['.' + group + '.data', '.' + group + '.lammpstrj']
)
for (nuc_topo, nuc_trj) in nuc_pairs:
    hns_nucleoid_cub_dist(
        nuc_topo,
        nuc_trj,
        lineage,
        save_to = save_to
    )